# Teste da API Comtrade

Info: https://comtradedeveloper.un.org

Specs: https://comtradedeveloper.un.org/api-details#api=comtrade-v1&operation=get-get



In [25]:
# %pip install requests

Note: you may need to restart the kernel to use updated packages.


In [26]:
# %pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
# %pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Ler os códigos HS

Obter em https://github.com/datasets/harmonized-system/blob/master/data/harmonized-system.csv

Copiar para directoria `support`


In [28]:
import pandas as pd

hs_codes_df = pd.read_csv('downloads/support/harmonized-system.csv')

Fazer um dicionário para os códigos de nível 2

In [29]:
hs_codes_l2 = hs_codes_df[hs_codes_df.level == 2]
hs_codes_l2[['hscode','description']]
hs_l2_map = dict(zip(hs_codes_l2.hscode, hs_codes_l2.description))


Aqui colocar os parâmetros gerais que não mudam.

In [30]:
baseUrl = "https://comtradeapi.un.org/public/v1"

m49_codes_cn_plp ={
    "024":"Angola",
    "076":"Brazil",
    "132":"Cabo Verde",
    "156":"China",
    "344":"China, Hong Kong",
    "446":"China, Macau",
    "226":"Guiné Equatorial",
    "624":"Guiné-Bissau",
    "508":"Moçambique",
    "620":"Portugal",
    "678":"São Tomé e Príncipe",
    "626":"Timor-Leste"
}
m49_china = "156"
m49_hong_kong = "344"
m49_macau = "446"
m49_plp = set(m49_codes_cn_plp.keys())-{m49_china,m49_hong_kong,m49_macau}
m49_plp_list = ",".join(m49_plp)

Aqui os parâmetros de cada chamada à API.

In [31]:
typeCode = "C" # C for commodities, S for Services
freqCode = "A" # A for annual and M for monthly
clCode = "HS" #Trade classifications Available values : HS, SS, B4, B5, EB, EB10, EB02, EBSDMX
reporterCode = m49_china # China, HK 344, Macau 446, see https://unstats.un.org/unsd/methodology/m49/
partnerCode = m49_plp_list
period="2021"
cmdCode="AG2" # TOTAL, AG2 (HS level 2)
flowCode = "M" # M=import, X=export,outros: RX, RM, MIP, XIP, MOP, XOP, MIF, XIF, DX, FM


In [32]:
import requests
requestUrl=f"{baseUrl}/preview/{typeCode}/{freqCode}/{clCode}"
resp = requests.get(requestUrl,
        {
        'reporterCode':m49_china,
        'period':period,
        'partnerCode':m49_plp_list,
        'cmdCode':cmdCode,
        'flowCode':flowCode
        },
        timeout=10)

In [33]:
import json
import pandas as pd

results = json.loads(resp.content)['data']
df = pd.DataFrame(results)
# convert result country codes to three-digit codes
df.reporterCode = df.reporterCode.astype('str').str.zfill(3)
df.partnerCode = df.partnerCode.astype('str').str.zfill(3)
# print(df.columns.values)

In [46]:


df.reporterDesc = df.reporterCode.map(m49_codes_cn_plp)
df.partnerDesc = df.partnerCode.map(m49_codes_cn_plp)
df.cmdDesc = df.cmdCode.map(hs_l2_map)

df['primaryValueFormated'] = df.primaryValue.map('{:,}'.format)
cols = ['typeCode','cmdCode','cmdDesc','freqCode','refYear','reporterDesc','partnerDesc','flowCode','primaryValueFormated','primaryValue']

pco = df[cols].sort_values(['partnerDesc','primaryValue'], ascending=[True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] < 6]

In [48]:
df[cols].to_excel(f"./downloads/dados_comtrade_{period}_{cmdCode}.xlsx")
pco.to_excel(f"./downloads/paises_cmd_ordenados_{period}.xlsx")
pco_top5.to_excel(f"./downloads/paises_cmd_top5_{period}.xlsx")